In [1]:
# Import modules
import sys

In [2]:
# Import custom modules
sys.path.append('../')

from src.models.upload_data import *

In [3]:
# Set notebook attributes
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Set magic attributes
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# Configuration
args = argparse.ArgumentParser()
args.add_argument("--config", default="../params.yaml")
parsed_args = vars(args.parse_args(""))
config_path = parsed_args['config']

### Till Activity Transaction

In [6]:
%%time
# Data ingestion, cleaning/wrangling
df_meta_trxn_raw = pull_data(config_path, metabase_sql(), 'DWH', 'metabase_trxn', False, 'raw')

Currently loading metabase_trxn data set ...


,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days
41164,7333380,2022-11-01,2022-11-04,2
95506,897271,2022-10-26,2022-11-24,29


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 125 ms
Wall time: 445 ms


In [7]:
%%time
# Data ingestion, cleaning/wrangling
df_meta_trxn_clean = clean_dataset(config_path, 'metabase_trxn')


Currently cleaning metabase_trxn data set ...


,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days
62185,7578059,2022-10-26,2022-11-24,29
1779,196777,2022-10-26,2022-11-22,11


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 1.69 s
Wall time: 4.63 s


In [8]:
%%time
# Data aggregation
df_meta_trxn_aggregate = meta_agg_summaries(config_path)

,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val
49300,7426900,2022-10-26,2022-11-24,29,22691.96
90789,7969709,2022-10-26,2022-11-24,26,33200.59


CPU times: total: 312 ms
Wall time: 891 ms


In [9]:
%%time
# Feature engineering
df_meta_trxn_features = meta_feature_engineering(config_path, extract_end_date)


Expected transaction days feature sample:
  store_number last_trx_date most_recent_trx_date_past_30_days  \
6       105714    2022-11-24                        2022-10-26   

   expected_trx_days  
6              30.00  
---------------------------------------------------------------------------------------------------------------------------------------

Consistency feature sample:
  store_number  expected_trx_days  actual_trx_days  page_active_days
6       105714                 30               29              0.97
---------------------------------------------------------------------------------------------------------------------------------------

Number of days since last transaction feature sample:
  store_number  days_since_last_trx last_trx_date
6       105714                 0.00    2022-11-24
---------------------------------------------------------------------------------------------------------------------------------------


,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency
2116,210823,2022-10-26,2022-11-24,29,96502.51,30,0.97,No_rules_relaxed,0.00,Yes,1.00
35858,7263692,2022-10-26,2022-11-16,19,100702.81,22,0.86,No_rules_relaxed,8.00,No,0.00


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 750 ms
Wall time: 2.14 s


In [10]:
# Final dataset schema
df_meta_trxn_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115150 entries, 0 to 115149
Data columns (total 11 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   store_number                       115150 non-null  object        
 1   most_recent_trx_date_past_30_days  99685 non-null   datetime64[ns]
 2   last_trx_date                      99685 non-null   datetime64[ns]
 3   actual_trx_days                    115150 non-null  int32         
 4   approx_30_days_trx_val             115150 non-null  float64       
 5   expected_trx_days                  115150 non-null  int32         
 6   page_active_days                   115150 non-null  float64       
 7   inference_col                      115150 non-null  object        
 8   days_since_last_trx                99685 non-null   float64       
 9   transacted_last_5_days             115150 non-null  object        
 10  weight_till_recency 